<a href="https://colab.research.google.com/github/Jodh/crossbar-synthesis/blob/master/Synthesis_of_convolution_layer_on_flowbased_xbar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. **Train and extract weights of a BNN for cifar-10**

2.   **Generate constraints for SMT solving for a given kernel (from the BNN)**

3.   **Solve the generated formula and map it to a crossbar**

4.   **Test the crossbar on cifar-10 test set using python functions to simulate the crossbar and the other layers**

5.   **The crossbar simulation function contains parameters to add errors**

6.   **Test cifar-10 test set on standard multiply accumulate crossbars also simulated using similar python functions with parameters to add errors**

Files needed: 

/content/cifar-10-bnn-model-30-80-10-weights.txt,

/content/cifar-test-flat.npy, 
 
/content/cifar-testY-flat.npy 





In [ ]:
!pip install z3-solver
!pip install lcapy

In [49]:
import numpy as np
import pickle
import os
import re
from lcapy import *
import time
import csv

**Helper Functions**

**Functions for generating the truth-table for a given kernel**

In [21]:
#helper functions       
def _hard_sigmoid(x):
    '''Hard sigmoid different from the more conventional form (see definition of K.hard_sigmoid).

    # Reference:
    - [BinaryNet: Training Deep Neural Networks with Weights and Activations Constrained to +1 or -1, Courbariaux et al. 2016](http://arxiv.org/abs/1602.02830}

    '''
    x = (0.5 * x) + 0.5
    return np.clip(x, 0, 1)

def binary_tanh(x):
    '''Binary hard sigmoid for training binarized neural network.
     The neurons' activations binarization function
     It behaves like the sign function during forward propagation
     And like:
        hard_tanh(x) = 2 * _hard_sigmoid(x) - 1 
        clear gradient when |x| > 1 during back propagation

    # Reference:
    - [BinaryNet: Training Deep Neural Networks with Weights and Activations Constrained to +1 or -1, Courbariaux et al. 2016](http://arxiv.org/abs/1602.02830}

    '''
    return 2 * np.round(_hard_sigmoid(x)) - 1

def batch_norm(x, weights):
    mean = weights[2]
    variance = weights[3]
    beta = weights[1]
    gamma = weights[0]
    epsilon = 0.000001
    y = (x-mean)/np.sqrt(variance+epsilon)
    return gamma*y+beta

def kernel_op(in_array,weights,parameters):
    w = np.sign(weights)
    conv2d_out = binary_tanh(batch_norm(np.dot(in_array,w),parameters))
    return conv2d_out
 
# take weights, dot product with input, normalize, acitvation function       
def fill_tt_deci():
    tt_deci = np.zeros((1,2),dtype=int)
    for i in range(2**5):
        tt_deci = np.append(tt_deci,np.array([[i,0]]),axis = 0)
    return tt_deci[1:,]
    
def fill_tt(tt_deci,weights,kernel):
    tt = np.zeros((2**5,6),dtype=bool)
    in_pixels = np.zeros((1,5),dtype=bool)
    w = weights[0][:,:,0,kernel][0]
    parameters = np.array((weights[1][kernel],weights[2][kernel],weights[3][kernel],weights[4][kernel]))
    # temp2 = np.zeros((1,13),dtype=bool)
    for i in range(2**5):
        in_pixels = (tt_deci[i,0] & (2**np.arange(5))>0)
        conv2d_out = kernel_op(in_pixels,w,parameters)
        if conv2d_out == -1:
            conv2d_out = 0
        tt[i,:] = np.append(in_pixels,conv2d_out)

    return tt

def generate_kernel_tt(kernel):
    with open("cifar-10-bnn-model-30-80-10-weights.txt", "rb") as fp:   # Unpickling
        weights = pickle.load(fp)
    tt_deci = fill_tt_deci()
    tt = fill_tt(tt_deci, weights, kernel)
    return tt

**Functions for gnerating constraints for the z3 solver**

In [39]:
# declare variables
def declare_variables():
    code = '(declare-fun M (Int Int) Int)\n'
    code += '(declare-fun I (Int Int) Bool)\n'
    code += '(declare-fun O (Int Int) Bool)\n'
    code += '(declare-fun m (Int Int Int) Bool)\n'
    code += '(declare-fun row (Int Int Int) Bool)\n'
    code += '(declare-fun col (Int Int Int) Bool)\n'
    
    return code

# assign values to variables, read from I/O truth table
def assign_values(n,m,input_size,truth_table,tt_size):
    # assign input
    code = '(assert (and\n'
    for k in range(tt_size):
        for b in range(input_size):
            code +=  '(= (I {k} {b}) {v})\n'.format(k=k,b=b,v=str(truth_table[k,b]).lower())
    # code += '\n'
    #assign output
    for k in range(tt_size):
        for b in range(1):
            code += '(= (O {k} {b}) {v})\n'.format(b=b,k=k,v=str(truth_table[k,b+input_size]).lower())
    #assign row zero to true for each input-row
    for k in range(tt_size):
        code += '(row 0 0 {k})\n'.format(k=k)
    #assign rest of the rows and columns at time 0 to false, row_itk, col_jtk
    for k in range(tt_size):
        for i in range(1,n):
            code += '(not (row {i} 0 {k}))\n'.format(i=i,k=k)
    for k in range(tt_size):
        for j in range(m):
            code += '(not (col {j} 0 {k}))\n'.format(j=j,k=k)
    code += '))\n'
    
    return code

#set range of memristor labels
def set_range_of_mem_labels(n,m,input_size):
    num_labels = 2*input_size+2
    code = '(assert (and\n'
    for i in range(n):
        for j in range(m):
            code += '(>= (M {i} {j}) 0) (<= (M {i} {j}) {v})\n'.format(i=i,j=j,v=num_labels-1)
    code += '))\n'
    return code

#set value of mijk
def assign_mijk(n,m,input_size,tt_size):
    code = '(assert (and\n'
    for k in range(tt_size):
        for i in range(n):
            for j in range(m):
                code += '(or\n'
                code += '(and (= (M {i} {j}) 0) (= (m {i} {j} {k}) false))\n'.format(i=i,j=j,k=k)
                code += '(and (= (M {i} {j}) 1) (= (m {i} {j} {k}) true))\n'.format(i=i,j=j,k=k)
                for label in range(input_size):
                    code += '(and (= (M {i} {j}) {labelPlus2}) (= (m {i} {j} {k}) (I {k} {label})))\n'.format(i=i,j=j,k=k,label=label,labelPlus2 = label+2)
                for not_label in range(input_size):
                    code += '(and (= (M {i} {j}) {labelPlus2}) (= (m {i} {j} {k}) (not (I {k} {label}))))\n'.format(i=i,j=j,k=k,label=not_label,labelPlus2 = not_label+2+input_size)
                code += ')\n'
    code += '))\n'
    return code

#set value of row and columns
def set_row_col(n,m,tt_size):
    time = n+m+1
    code = '(assert (and\n'
    for k in range(tt_size):
        for t in range(1,time):
            for j in range(m):
                code += '(= (col {j} {t} {k}) (or (col {j} {tminus1} {k})\n'.format(j=j,k=k,t=t,tminus1=t-1)
                for i in range(n):
                    code += '(and (row {i} {tminus1} {k}) (m {i} {j} {k}))\n'.format(i=i,j=j,tminus1=t-1,k=k)
                code += '))\n'
            for i in range(n):
                code += '(= (row {i} {t} {k}) (or (row {i} {tminus1} {k})\n'.format(i=i,k=k,t=t,tminus1=t-1)
                for j in range(m):
                    code += '(and (col {j} {tminus1} {k}) (m {i} {j} {k}))\n'.format(i=i,j=j,tminus1=t-1,k=k)
                code += '))\n'
    code += '))\n'
    return code

#check rowntk against output
def check_output(n,m,tt_size):
    code = '(assert (and\n'
    for k in range(tt_size):
        code += '(= (row {r} {T} {k}) (O {k} 0))\n'.format(r=n-1,T=n+m,k=k)
    code += '))\n'
    return code
# check solvability
def footer(n,m):
    code = '(check-sat)\n'
#    for i in range(n):
#        for j in range(m):
#            code += '(get-value ((M {i} {j})))'.format(i=i,j=j)
    # code += '(get-model)\n'
    return code
#debug-functions
def print_stuff(n,m,tt_size):
    code = ''
    for i in range(n):
        for j in range(m):
            code += '(get-value ((M {i} {j})))\n'.format(i=i,j=j)
    for k in range(tt_size):
        # for b in range(5):
        code += '(get-value ((row {b} {T} {k})))\n'.format(b=n-1,T=n+m,k=k)
    return code

#n=int(argn)
#m=int(argm)
#kernel = int(arg_kernel)

def write_formula(n,m,kernel):
    truth_table = generate_kernel_tt(kernel)
    input_size = truth_table.shape[1]-1
    tt_size = truth_table.shape[0]
    filepath = 'formulas/'
    filename = 'kernel-{n}x{m}-{b}.txt'.format(n=n,m=m,b = kernel)
    with open(filepath+filename,'w') as writer:
        writer.write(declare_variables())
        writer.write(assign_values(n, m, input_size, truth_table, tt_size))
        writer.write(set_range_of_mem_labels(n, m, input_size))
        writer.write(assign_mijk(n, m, input_size, tt_size))
        writer.write(set_row_col(n, m, tt_size))
        writer.write(check_output(n, m, tt_size))
        writer.write(footer(n,m))
        writer.write(print_stuff(n, m, tt_size))


**Create formulas and designs directory**

In [40]:
from pathlib import Path
d = Path("/content/designs")
f = Path("/content/formulas")
if not(d.exists()):
  !mkdir ./designs
if not(f.exists()):
  !mkdir ./formulas


**Generate fromulas and solve them for designs**

In [41]:
# range is number of kernels in your convolution layer, 30 in our case.
for kernel in range(30):
#    truth_table = generate_kernel_tt(kernel)       
    write_formula(5, 5,kernel)

In [42]:
n = 5
m = 5
path = '/content/'
for i in range(30):
    command = 'z3 {path}formulas/kernel-{n}x{m}-{i}.txt > {path}designs/design-kernel-{n}x{m}-{i}.txt'.format(i=i, n=n, m=m, path=path)
#    command = 'z3 {path}formulas/kernel-{n}x{m}-{i}.txt'.format(i=i,n=n,m=m,path=path)
    # os.system(command)
    !{command}
    print('kernel design {i} found...'.format(i=i))

kernel design 0 found...
kernel design 1 found...
kernel design 2 found...
kernel design 3 found...
kernel design 4 found...
kernel design 5 found...
kernel design 6 found...
kernel design 7 found...
kernel design 8 found...
kernel design 9 found...
kernel design 10 found...
kernel design 11 found...
kernel design 12 found...
kernel design 13 found...
kernel design 14 found...
kernel design 15 found...
kernel design 16 found...
kernel design 17 found...
kernel design 18 found...
kernel design 19 found...
kernel design 20 found...
kernel design 21 found...
kernel design 22 found...
kernel design 23 found...
kernel design 24 found...
kernel design 25 found...
kernel design 26 found...
kernel design 27 found...
kernel design 28 found...
kernel design 29 found...


**Functions for parsing design outputs from z3 and mapping to xbar and evaluatiing the xbar output** [work needed to check or improve the circuit simulation, can be replaced with a pspice simulation instead]

In [37]:
def convert_file_to_matrix(filename,n,m):
    matrix = np.zeros((n,m),dtype=int)
    file = open(filename,'r')
    lines = file.readlines()[1:]
    for i in range(n):
        for j in range(m):
            line = n*i+j
            x = re.search("\(*M.{6}([\w]{1,2})\)*", lines[line])
            matrix[i,j] = int(x.group(1))
    
    return matrix

def find_mem_states(D, d_input):
#    t0 = time.time()
    num_literals = 2*len(d_input)+2
    mapping = np.zeros(num_literals, dtype='bool')
    m = np.zeros(np.shape(D))
    for i in range(num_literals):
        if i <=1:
            mapping[i] = i
        elif i <= (len(d_input)+1):
            mapping[i]=d_input[i-2]
            mapping[i+len(d_input)] = not(d_input[i-2])
    for i in range(len(D[:,1])):
        for j in range(len(D[1,:])):
            m[i,j] = mapping[D[i,j]]
#    t1 = time.time()
#    print('finding mem states',t1-t0)
    return m

def flow_at_outwire(mem_states):
#    t0 = time.time()
    n = len(mem_states[:,1])
    l = len(mem_states[1,:])
    r = np.zeros(n,dtype='bool')
    r[0] = 1
    c = np.zeros(l,dtype='bool')
    # iteration through time
    t = 0
    while t <= 10:
        for j in range(l):
            for i in range(n):
                c[j] = c[j] or (mem_states[i,j] and r[i])
        for i in range(1,n):
            for j in range(l):
                r[i] = r[i] or (mem_states[i,j] and c[j])

        t = t+1
#    t1 = time.time()
#    print('flow finding',t1-t0)

    return r[4]

def drift_error(resistor, min_err,max_err):
    resistor = int(resistor)
    res_max = 20000
    min_err = res_max*min_err/100
    max_err = res_max*max_err/100
    noise = np.random.randint(0,max_err)
    return resistor+noise

def drift_error_mac(resistor, min_err,max_err):
    noise = (max_err/100 - min_err/100) * np.random.random_sample() - (min_err/100)
    return resistor+noise
    
    
def flow_based(n,m,err):
    total_missmatch = 0
    name_prefix = '{n}x{m}-'.format(n=n,m=m)
    for kernel in range(30):
        truth_table = generate_kernel_tt(kernel)
        kernel_name = 'designs/design-kernel-'+name_prefix+str(kernel)+'.txt'
        D = convert_file_to_matrix(kernel_name,n,m)
#        mem_state = find_mem_states(D, truth_table[1])
        true_min = 1
        false_max = 0
        miss_matches = 0
        
        for row in range(2**5):

#            for trial in range(100):
            mem_state = find_mem_states(D, truth_table[row][:5])
#            true_out = flow_at_outwire(mem_state)
            true_out = truth_table[row,5]
            for trial in range(1):
                ckt = Circuit()
                ckt.add('Vin r1 0 0.3')
                for i in range(n):
                    for j in range(m):
#                        programming_err = np.random.random(1)
                        if mem_state[i,j] == 1:
                            resistor = '2000'
                        else:
                            resistor = '600000'
                        code = 'R{i}{j} r{i} c{j} {v}'.format(i=i+1,j=j+1,v=drift_error(resistor,-err,err))
#                        code = 'R{i}{j} r{i} c{j} {v}'.format(i=i+1,j=j+1,v=resistor)
                        ckt.add(code)
                ckt.add('Rout r{n} 0 100'.format(n=n))
#                acc[row] += int(ckt.Rout.i.evaluate()>=0.00259)
#            print(row, acc[row])
                current = ckt.Rout.i.evaluate()
#                print(current,true_out)
                if (current >= 5.5e-6) != true_out:
                    miss_matches +=1
                if true_out:
                    if true_min >  current:
                        true_min = current
                else:
                    if false_max < current:
                        false_max = current
#                print(ckt.Rout.i.evaluate(), true_out)
#        print(miss_matches)
#        print(false_max,true_min)
        total_missmatch += miss_matches
    print(total_missmatch)
    return '{res_err},{out_err}\n'.format(res_err=err, out_err=total_missmatch*100/960)

def batch_norm(x, weights):
    mean = weights[2]
    variance = weights[3]
    beta = weights[1]
    gamma = weights[0]
    epsilon = 0.000001
    y = (x-mean)/np.sqrt(variance+epsilon)
    return gamma*y+beta

def _hard_sigmoid(x):
    x = (0.5 * x) + 0.5
    return np.clip(x, 0, 1)

def binary_tanh(x):
    return 2 * np.round(_hard_sigmoid(x)) - 1

def ADC(current):
    current = 1000*current
    A = 5.4
    M = 11
    value = (2/M)*round((M/2)*current)
    return np.round(A*value)
    
def mac_based(err):
    with open("cifar-10-bnn-model-30-80-10-weights.txt", "rb") as fp:   # Unpickling
        weights = pickle.load(fp)
    total_missmatches = 0
    
    for kernel in range(30):
        truth_table = generate_kernel_tt(kernel)
        missmatches = 0
        w = drift_error_mac(np.sign(drift_error_mac(weights[0][:,:,0,kernel][0],-err,err)), -err,err)
#        w = np.sign(weights[0][:,:,0,kernel][0])
        masking_array = np.where(w == 1, 1, 0)
        masking_array2 = np.where(w == -1, 1, 0)
#        print(w, masking_array, masking_array2)
        parameters = np.array((weights[1][kernel],weights[2][kernel],weights[3][kernel],weights[4][kernel]))
        
        for row in range(2**5):
            true_out = truth_table[row,5]
            v = truth_table[row,0:5]*0.1
#            v_real = truth_table[row,0:5]*1
            v1 = v*masking_array
            v2 = v*masking_array2
#            print(v,v1,v2)
            ckt = Circuit()
            ckt2 = Circuit()

            ckt.add('V0 r0 0 {v0}'.format(v0 = v1[0]))
            ckt.add('V1 r1 0 {v1}'.format(v1 = v1[1]))
            ckt.add('V2 r2 0 {v2}'.format(v2 = v1[2]))
            ckt.add('V3 r3 0 {v3}'.format(v3 = v1[3]))
            ckt.add('V4 r4 0 {v4}'.format(v4 = v1[4]))
            for i in range(5):
                ckt.add('R{i} r{i} plus 100'.format(i=i))
            ckt.add('Rout plus 0 100')
#            
            ckt2.add('V0 r0 0 {v0}'.format(v0 = v2[0]))
            ckt2.add('V1 r1 0 {v1}'.format(v1 = v2[1]))
            ckt2.add('V2 r2 0 {v2}'.format(v2 = v2[2]))
            ckt2.add('V3 r3 0 {v3}'.format(v3 = v2[3]))
            ckt2.add('V4 r4 0 {v4}'.format(v4 = v2[4]))
            for i in range(5):
                ckt2.add('R{i} r{i} plus 100'.format(i=i))
            ckt2.add('Rout plus 0 100')
            total_current = ADC(ckt.Rout.i.evaluate()- ckt2.Rout.i.evaluate())
#            out_bit = binary_tanh(batch_norm(np.dot(v_real,w),parameters))
#            if out_bit == -1:
#                out_bit = 0
#            print(ADC(total_current))
#            for i in range(5):
#                if w[i] == 1:
#                    current_pos += currents[i]
#                else:
#                    current_neg += currents[i]
#            print(ckt_pos.Rout.i.evaluate(), ckt_neg.Rout.i.evaluate())
#            total_current = (ckt_pos.Rout.i.evaluate() - 0.2*np.sum(truth_table[row,0:5]))
#            print(ckt_pos.Rout.v.evaluate(), ckt_neg.Rout.v.evaluate())
#            print(total_current,true_out)
            mac_out = binary_tanh(batch_norm(total_current,parameters))
            if mac_out == -1:
                mac_out = 0
##            print(round(total_current*1e5),mac_out, true_out)
#            if out_bit != true_out:
            if mac_out != true_out:
                missmatches += 1
##        print(missmatches)
        total_missmatches += missmatches
    print(total_missmatches)
    return '{res_err},{out_err}\n'.format(res_err=err, out_err=total_missmatches*100/960)

**Run the flow based and multiply accumulate models using the above functions on various drift values and store error rates**

compares the output of the circuits with the output of the kernels using the BNN weight file

these error rates are used as computational short-hand for simulating errors in the inference phase in the next code block. Saves time on actually simulating the circuit for every input.

In [44]:
# drift_values = [1,2,3,4,5,6,7,8,9,10]
drift_values = [1]

with open('flow-error.csv','w') as file:
   for i in drift_values:
       file.write(flow_based(5,5,i))
       print(i, 'done...')
        
with open('mac-error.csv','w') as file:
    for i in drift_values:
        file.write(mac_based(i))
        print(i, 'done...')

51
1 done...
235
1 done...


**Functions for testing accuracy of whole BNN model by replacing the convolution layer with above simulated circuits**

In [52]:
images = np.load('cifar-test-flat.npy')
#helper functions       
def drift_error(weights, a,b):
#    noise = (b-a)*np.random.random_sample()-a
#    weights = weights + np.multiply(weights,noise)/100
    res_max = 0.5
    a = res_max*a/100
    b = res_max*b/100
    noise = (b-a)*np.random.random_sample(np.shape(weights))-a
    return weights+noise

def drift_error_parameters(weights, a,b):
    noise = (b-a)*np.random.random_sample(np.shape(weights))-a
    weights = weights + np.multiply(weights,noise)/100
#    res_max = 0.5
#    a = res_max*a/100
#    b = res_max*b/100
#    noise = np.random.randint(a,b)
    return weights

#  KERNEL function input: image, weights[0:5], output: -1,+1 array of 30*200
def conv2d_crossbar(image,num_kernels,weights):
    window_size = 5
    conv2d_out = np.zeros((image.shape[0],num_kernels))
    #pad input
    padded_image = np.zeros(image.shape[0]+4)
    padded_image[2:image.shape[0]+2] = image
    #for each kernel shift over windows and do convolution
    for kernel in range(num_kernels):
#        w = np.sign(weights[0][:,:,0,kernel][0])
#        parameters = np.array((weights[1][kernel],weights[2][kernel],weights[3][kernel],weights[4][kernel]))
        kernel_name = 'designs/design-kernel-'+str(kernel)+'.txt'
        D = convert_file_to_matrix(kernel_name,5,5)
        for i in range(image.shape[0]):
#            conv2d_out[i,kernel] = binary_tanh(batch_norm(np.dot(padded_image[i:i+window_size],w),parameters))
#            conv2d_out[kernel*image.shape[0]+i] = batch_norm(np.dot(padded_image[i:i+window_size],w),parameters)
#            conv2d_out[i,kernel] = np.dot(padded_image[i:i+window_size],w)
            
            mem_states = find_mem_states(D, padded_image[i:i+window_size])
            conv2d_out[i,kernel] = np.sign(flow_at_outwire(mem_states)-0.1) 
    return conv2d_out.reshape(num_kernels*image.shape[0])

def conv2d(image,num_kernels,weights, error, err):
    window_size = 5
    conv2d_out = np.zeros((image.shape[0],num_kernels))
    #pad input
    padded_image = np.zeros(image.shape[0]+4)
    padded_image[2:image.shape[0]+2] = image
    #for each kernel shift over windows and do convolution
    for kernel in range(num_kernels):
        w = np.sign(weights[0][:,:,0,kernel][0])
        parameters = np.array((weights[1][kernel],weights[2][kernel],weights[3][kernel],weights[4][kernel]))
        for i in range(image.shape[0]):
            out_bit = binary_tanh(batch_norm(np.dot(padded_image[i:i+window_size],w),parameters))
            if np.random.random() < err/100 and error:
                out_bit = -1*out_bit
            conv2d_out[i,kernel] = out_bit
#            conv2d_out[kernel*image.shape[0]+i] = batch_norm(np.dot(padded_image[i:i+window_size],w),parameters)
#            conv2d_out[i,kernel] = np.dot(padded_image[i:i+window_size],w)
    return conv2d_out.reshape(num_kernels*image.shape[0])

def conv2d_MAC(image,num_kernels,weights, error, err_range):
    window_size = 5
    conv2d_out = np.zeros((image.shape[0],num_kernels))
    #pad input
    padded_image = np.zeros(image.shape[0]+4)
    padded_image[2:image.shape[0]+2] = image
    #for each kernel shift over windows and do convolution
    for kernel in range(num_kernels):
        w = drift_error(np.sign(weights[0][:,:,0,kernel][0]), -err_range,err_range)
        parameters = drift_error_parameters(np.array((weights[1][kernel],weights[2][kernel],weights[3][kernel],weights[4][kernel])),-err_range,err_range)
        for i in range(image.shape[0]):
            out_bit = binary_tanh(batch_norm(np.dot(padded_image[i:i+window_size],w),parameters))
            conv2d_out[i,kernel] = out_bit
#            conv2d_out[kernel*image.shape[0]+i] = batch_norm(np.dot(padded_image[i:i+window_size],w),parameters)
#            conv2d_out[i,kernel] = np.dot(padded_image[i:i+window_size],w)
    return conv2d_out.reshape(num_kernels*image.shape[0])

def conv2d_MAC_error_model(image,num_kernels,weights, error, err):
    window_size = 5
    conv2d_out = np.zeros((image.shape[0],num_kernels))
    #pad input
    padded_image = np.zeros(image.shape[0]+4)
    padded_image[2:image.shape[0]+2] = image
    #for each kernel shift over windows and do convolution
    for kernel in range(num_kernels):
        w = np.sign(weights[0][:,:,0,kernel][0])
        parameters = np.array((weights[1][kernel],weights[2][kernel],weights[3][kernel],weights[4][kernel]))
        for i in range(image.shape[0]):
            out_bit = binary_tanh(batch_norm(np.dot(padded_image[i:i+window_size],w),parameters))
            if np.random.random() < err/100 and error:
                out_bit = -1*out_bit
            conv2d_out[i,kernel] = out_bit
#            conv2d_out[kernel*image.shape[0]+i] = batch_norm(np.dot(padded_image[i:i+window_size],w),parameters)
#            conv2d_out[i,kernel] = np.dot(padded_image[i:i+window_size],w)
    return conv2d_out.reshape(num_kernels*image.shape[0])

# DENSE LAYER FUNCTIONS
def dense_layer_exact_1(conv2d, num_nodes, weights, layer_num, err_range):
    dense_out = np.zeros(num_nodes)
    for node in range(num_nodes):
        a = drift_error(binary_tanh(weights[layer_num][:,node]),-err_range,err_range)
        parameters = drift_error_parameters(np.array((weights[layer_num+1][node],weights[layer_num+2][node],weights[layer_num+3][node],weights[layer_num+4][node])), -err_range,err_range)
        dense_out[node] = binary_tanh(batch_norm(np.dot(conv2d,a),parameters))
#        dense_out[node] = batch_norm(np.dot(conv2d,a),parameters)
#        dense_out[node] = np.dot(conv2d,a)
    return dense_out

def dense_layer_exact_out(dense_1, num_nodes, weights, layer_num, err_range):
    dense_out = np.zeros(num_nodes)
    for node in range(num_nodes):
        a = drift_error(binary_tanh(weights[layer_num][:,node]),-err_range,err_range)
        parameters = drift_error_parameters(np.array((weights[layer_num+1][node],weights[layer_num+2][node],weights[layer_num+3][node],weights[layer_num+4][node])), -err_range, err_range)
#        dense_out[node] = binary_tanh(batch_norm(np.dot(conv2d,a),parameters))
        dense_out[node] = batch_norm(np.dot(dense_1,a),parameters)
    return dense_out

def classify_cifar(conv_kernels, dense_1_nodes, dense_2_nodes, err_row0, err_row1, hybrid=True):
    matches = 0
    with open("cifar-10-bnn-model-30-80-10-weights.txt", "rb") as fp:   # Unpickling
        weights = pickle.load(fp)
    images = np.load('cifar-test-flat.npy')
#    images = images[0:1000]
    y_test = np.load('cifar-testY-flat.npy')
#    t0 = time.time()
    for i in range(500):
#        conv2d_out = conv2d_crossbar(images[i],30,weights)
        if hybrid:
            conv2d_out = conv2d(images[i],30,weights,1, err_row1)
        else:
           conv2d_out = conv2d_MAC_error_model(images[i],30,weights,1, err_row0)
            # conv2d_out = conv2d_MAC(images[i],30,weights,1, err_row0)
        dense_1 = dense_layer_exact_1(conv2d_out, dense_1_nodes, weights, 5, 0)
        dense_2 = dense_layer_exact_out(dense_1, dense_2_nodes, weights, 10, 0)
        y_out = np.argmax(dense_2)
        if np.argmax(y_test[i]) == y_out:
            matches +=1
#        if i % 1000 == 0:
#            if i == 0:
#                continue
#            else:
#                t1 = time.time()
#                print(i,'image mark....',t1-t0)
#                print(100*(matches/(i)))
    print(err_row0,100*(matches/500))
    return '{},{}\n'.format(err_row0,100*(matches/500))


def test_error_rates(hybrid):
    out_string = ''
    model_type = ''
    if hybrid:
      model_type = 'flow-error.csv'
    else:
      model_type = 'mac-error.csv'
    with open(model_type, mode='r') as file:
    # with open('flow-error.csv',mode = 'r') as file: #temp, comment out later
        reader = csv.reader(file, delimiter=',')
        for row in reader:            
            out_string += classify_cifar(30, 80, 10, float(row[0]),float(row[1]),hybrid)
    return out_string

**Flow Based Model**

In [50]:
with open('drift-vs-hybrid-accuracy.csv','w') as file:
    file.write(test_error_rates(True))


1.0 88.8


**Multiply Accumilate Model**

In [53]:
with open('drift-vs-mac-accuracy.csv','w') as file:
    file.write(test_error_rates(False))

1.0 92.0
